In [1]:
#读数据,数据变形
import pandas as pd 
import numpy as np
from keras.utils import to_categorical
from keras.applications.imagenet_utils import preprocess_input

Train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv').copy()
X_train = Train.drop(columns=['label'])
y_train = Train['label']

X_train = np.array(X_train)
X_train = X_train.reshape((42000,28,28,1))

X_test = np.array(X_test)
X_test = X_test.reshape((28000,28,28,1))

X_train = X_train/255
X_test = X_test/255
#X_train = preprocess_input(np.array(X_train))
#X_test = preprocess_input(np.array(X_test))

y_train = to_categorical(y_train,num_classes=10)

print(X_train.shape,X_test.shape,y_train.shape)

C:\Users\Elane053\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(42000, 28, 28, 1) (28000, 28, 28, 1) (42000, 10)


In [5]:
#构建模型
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,MaxPooling2D,Flatten
from keras.utils import np_utils, to_categorical

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Dropout(0.5))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
dropout_4 (Dropout)          (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
dropout_5 (Dropout)          (None, 11, 11, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
__________

In [6]:
#训练
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

num_epochs = 30
BatchSize = 128
#保存最好的模型
tensorboard = TensorBoard(log_dir='log')
checkpoint = ModelCheckpoint(filepath='best.h5',monitor='val_acc',mode='auto' ,save_best_only='True')
callback_lists=[tensorboard,checkpoint]

model.fit(X_train, y_train, epochs=num_epochs, batch_size=BatchSize,validation_split=0.1,callbacks=callback_lists)

Train on 37800 samples, validate on 4200 samples
Epoch 1/30
37800/37800 [==============================] - 120s 3ms/step - loss: 0.4337 - acc: 0.8590 - val_loss: 0.2915 - val_acc: 0.9721
Epoch 2/30
37800/37800 [==============================] - 116s 3ms/step - loss: 0.1269 - acc: 0.9615 - val_loss: 0.1978 - val_acc: 0.9726
Epoch 3/30
37800/37800 [==============================] - 119s 3ms/step - loss: 0.0980 - acc: 0.9699 - val_loss: 0.1407 - val_acc: 0.9869
Epoch 4/30
37800/37800 [==============================] - 117s 3ms/step - loss: 0.0771 - acc: 0.9757 - val_loss: 0.1058 - val_acc: 0.9862
Epoch 5/30
37800/37800 [==============================] - 135s 4ms/step - loss: 0.0704 - acc: 0.9778 - val_loss: 0.1144 - val_acc: 0.9862
Epoch 6/30
37800/37800 [==============================] - 125s 3ms/step - loss: 0.0630 - acc: 0.9802 - val_loss: 0.0746 - val_acc: 0.9890
Epoch 7/30
37800/37800 [==============================] - 124s 3ms/step - loss: 0.0603 - acc: 0.9805 - val_loss: 0.0799 - v

In [7]:
from keras.models import load_model
import pandas as pd
#model = load_model('best.h5')
#model.fit(X_train, y_train, epochs=20, batch_size=BatchSize,validation_split=0.1,callbacks=callback_lists)
model = load_model('best.h5')
y_pre = model.predict(X_test,verbose=1)
y_output = np.argmax(y_pre,axis=1)

sample = pd.read_csv('sample_submission.csv')
index = sample['ImageId']

output = pd.DataFrame()
output['ImageId'] = index
output['Label'] = y_output
print(output.head())
output.to_csv('result.csv',index=False)


Train on 37800 samples, validate on 4200 samples
Epoch 1/20
37800/37800 [==============================] - 140s 4ms/step - loss: 0.0328 - acc: 0.9897 - val_loss: 0.0395 - val_acc: 0.9929
Epoch 2/20
37800/37800 [==============================] - 163s 4ms/step - loss: 0.0327 - acc: 0.9897 - val_loss: 0.0424 - val_acc: 0.9907
Epoch 3/20
37800/37800 [==============================] - 140s 4ms/step - loss: 0.0332 - acc: 0.9890 - val_loss: 0.0382 - val_acc: 0.9926
Epoch 4/20
37800/37800 [==============================] - 151s 4ms/step - loss: 0.0317 - acc: 0.9902 - val_loss: 0.0378 - val_acc: 0.9943
Epoch 5/20
37800/37800 [==============================] - 143s 4ms/step - loss: 0.0301 - acc: 0.9901 - val_loss: 0.0331 - val_acc: 0.9933
Epoch 6/20
37800/37800 [==============================] - 136s 4ms/step - loss: 0.0295 - acc: 0.9908 - val_loss: 0.0370 - val_acc: 0.9924
Epoch 7/20
37800/37800 [==============================] - 140s 4ms/step - loss: 0.0286 - acc: 0.9912 - val_loss: 0.0341 - v